In [1]:
import queue

train = '''Player 1:
9
2
6
3
1

Player 2:
5
8
4
7
10'''

test = '''Player 1:
45
10
43
46
25
36
16
38
30
15
26
34
9
2
44
1
4
40
5
24
49
3
41
19
13

Player 2:
28
50
37
20
6
42
32
47
39
22
14
7
21
17
27
8
48
11
23
12
18
35
29
33
31'''

class Combat:
    
    def __init__(self, deck_1, deck_2):

        self.winner = None
        
        self.decks = {
            1: collections.deque(),
            2: collections.deque()}
        
        self.decks[1].extend(deck_1)
        self.decks[2].extend(deck_2)
        
    def play_round(self):
        
        if len(self.decks[1]) == 0:
            self.winner = 2
            raise ValueError('Game Over')
        elif len(self.decks[2]) == 0:
            self.winner = 1
            raise ValueError('Game Over')
        
        player_1_plays = self.decks[1].popleft()
        player_2_plays = self.decks[2].popleft()
                  
        if player_1_plays > player_2_plays:
            self.decks[1].append(player_1_plays)
            self.decks[1].append(player_2_plays)
        elif player_1_plays < player_2_plays:
            self.decks[2].append(player_2_plays)
            self.decks[2].append(player_1_plays)
        else:
            raise NotImplementedError
            
        return self
            
    def finish_game(self):
        while True:
            try:
                self.play_round()
            except ValueError:
                break
                
        return self
        
    def calc_winning_score(self):
        
        scores = {
            player_id: pd.Series(
                data  = list(deck),
                index = list(range(len(deck), 0, -1)))
            for player_id, deck
            in self.decks.items()}
    
        return {
            player_id: (score.index * score).sum() if len(score) > 0 else 0.0
            for player_id, score
            in scores.items()}[self.winner]
    
    @classmethod
    def setup(cls, starting_decks):
        
        decks = {
            1: [],
            2: []}
        
        for line in starting_decks.split('\n'):
            
            if len(line) == 0:
                continue
            elif line.startswith('Player '):
                player_id = int(line[len('Player '): -1])
            else:
                decks[player_id].append(int(line))
                
        return cls(deck_1=decks[1], deck_2=decks[2])
                
self = Combat.setup(starting_decks=train)
assert self.finish_game().calc_winning_score() == 306

self = Combat.setup(starting_decks=test)
self.finish_game().calc_winning_score()

/home/ec2-user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:122: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


33400

In [2]:
import tqdm
import sys

class RecursiveCombat(Combat):
    
    last_game_id = 0
    bar          = tqdm.tqdm()
    
    def __init__(self, deck_1, deck_2):

        RecursiveCombat.last_game_id += 1
        
        self.game_id               = RecursiveCombat.last_game_id
        self.configuration_history = set()
        
        super().__init__(deck_1=deck_1, deck_2=deck_2)

    def play_round(self):
        
        if len(self.decks[1]) == 0:
            self.winner = 2
            raise ValueError('Game Over')
        elif len(self.decks[2]) == 0:
            self.winner = 1
            raise ValueError('Game Over')

        configuration = (
            tuple(self.decks[1]),
            tuple(self.decks[2]))
        
        if configuration in self.configuration_history:
            self.winner = 1
            raise ValueError('Game Over')
        else:
            self.configuration_history.add(configuration)
            
        player_1_plays = self.decks[1].popleft()
        player_2_plays = self.decks[2].popleft()
        
        if ((len(self.decks[1]) >= player_1_plays)
        and (len(self.decks[2]) >= player_2_plays)):
            
            winner = RecursiveCombat.get_winner(
                deck_1  = tuple(list(self.decks[1])[:player_1_plays]),
                deck_2  = tuple(list(self.decks[2])[:player_2_plays]))
            
            if winner == 1:
                self.decks[1].append(player_1_plays)
                self.decks[1].append(player_2_plays)
            elif winner == 2:
                self.decks[2].append(player_2_plays)
                self.decks[2].append(player_1_plays)
            else:
                raise NotImplementedError
            
        else:

            if player_1_plays > player_2_plays:
                self.decks[1].append(player_1_plays)
                self.decks[1].append(player_2_plays)
            elif player_1_plays < player_2_plays:
                self.decks[2].append(player_2_plays)
                self.decks[2].append(player_1_plays)
            else:
                raise NotImplementedError
                
    @classmethod
    @functools.lru_cache(maxsize=sys.maxsize)
    def get_winner(cls, deck_1, deck_2):
        
        self.bar.update()        
        game = RecursiveCombat(deck_1=deck_1, deck_2=deck_2)

        # print(f'=== Game {game.game_id} ===')
        # print(f"Player 1's deck: {', '.join(map(str, deck_1))}")
        # print(f"Player 2's deck: {', '.join(map(str, deck_2))}")
        
        game.finish_game()
        
        # print(f'The winner of game {game.game_id} is player {game.winner}!')
        
        return game.winner
                
self = RecursiveCombat.setup(starting_decks=train)
assert self.finish_game().calc_winning_score() == 291

self = RecursiveCombat.setup(starting_decks=test)
self.finish_game().calc_winning_score()

0it [00:00, ?it/s]/home/ec2-user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:122: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
10906it [00:02, 4486.21it/s]

33745